In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
incidents = pd.read_csv('../input/candev/INCIDENTS_NEW.CSV')

In [ ]:
import holidays
Canada_holidays = holidays.Canada()
def weekend_or_not(date):
    if date.weekday() > 4:
        return True
    else:
        return False


from datetime import timedelta
def difference_dates(date1,date2):
    delta = date2-date1
    all_dates = []
    for i in range(delta.days+1):
        day = date2 + timedelta(days=i)
        all_dates.append(day)
    holiday_count=0
    for date in all_dates:
        stat_holiday = date in Canada_holidays
        if weekend_or_not(date)==True or stat_holiday==True:
            holiday_count+=1
        else:
            pass
    holidays_seconds = holiday_count * 24 * 60 * 60
    delta = delta.total_seconds()-holidays_seconds
    return float(delta/3600) 

In [ ]:
incident_history['CHANGE_DATE'] = pd.to_datetime(incident_history['CHANGE_DATE'], format='%d%b%Y:%H:%M:%S')

In [ ]:
incidents["ACTUAL_COMPLETION_HRS"].mean()

In [ ]:
incidents["BUSINESS_COMPLETION_HRS"].mean()

In [ ]:
subset = incidents[(incidents["EXTERNAL_SYSTEM"]!="EVENTMANAGEMENT")]

In [ ]:
subset["ACTUAL_COMPLETION_HRS"].mean()

In [ ]:
subset["BUSINESS_COMPLETION_HRS"].mean()

In [ ]:
subset.to_csv("./incidents_noEventManagement.csv") 

In [ ]:
import datetime


class BusinessHours:

    def __init__(self, datetime1, datetime2, worktiming=[9, 17],
                 weekends=[6, 7]):
        self.weekends = weekends
        self.worktiming = worktiming
        self.datetime1 = datetime1
        self.datetime2 = datetime2
        self.day_hours = (self.worktiming[1]-self.worktiming[0])
        self.day_minutes = self.day_hours * 60 # minutes in a work day

    def getdays(self):
        return int(self.getminutes() / self.day_minutes)
    
    def gethours(self):
        return int(self.getminutes() / 60)

    def getminutes(self):
        """
        Return the difference in minutes.
        """
        # Set initial default variables
        dt_start = self.datetime1  # datetime of start
        dt_end = self.datetime2    # datetime of end
        worktime_in_seconds = 0

        if dt_start.date() == dt_end.date():
            # starts and ends on same workday
            full_days = 0
            if self.is_weekend(dt_start):
                return 0
            else:
                if dt_start.hour < self.worktiming[0]:
                    # set start time to opening hour
                    dt_start = datetime.datetime(
                        year=dt_start.year,
                        month=dt_start.month,
                        day=dt_start.day,
                        hour=self.worktiming[0],
                        minute=0)
                if dt_start.hour >= self.worktiming[1] or \
                        dt_end.hour < self.worktiming[0]:
                    return 0
                if dt_end.hour >= self.worktiming[1]:
                    dt_end = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[1],
                        minute=0)
                worktime_in_seconds = (dt_end-dt_start).total_seconds()
        elif (dt_end-dt_start).days < 0:
            # ends before start
            return 0
        else:
            # start and ends on different days
            current_day = dt_start  # marker for counting workdays
            while not current_day.date() == dt_end.date():
                if not self.is_weekend(current_day):
                    if current_day == dt_start:
                        # increment hours of first day
                        if current_day.hour < self.worktiming[0]:
                            # starts before the work day
                            worktime_in_seconds += self.day_minutes*60  # add 1 full work day
                        elif current_day.hour >= self.worktiming[1]:
                            pass  # no time on first day
                        else:
                            # starts during the working day
                            dt_currentday_close = datetime.datetime(
                                year=dt_start.year,
                                month=dt_start.month,
                                day=dt_start.day,
                                hour=self.worktiming[1],
                                minute=0)
                            worktime_in_seconds += (dt_currentday_close
                                         - dt_start).total_seconds()
                    else:
                        # increment one full day
                        worktime_in_seconds += self.day_minutes*60
                current_day += datetime.timedelta(days=1)  # next day
            # Time on the last day
            if not self.is_weekend(dt_end):
                if dt_end.hour >= self.worktiming[1]:  # finish after close
                    # Add a full day
                    worktime_in_seconds += self.day_minutes*60
                elif dt_end.hour < self.worktiming[0]:  # close before opening
                    pass  # no time added
                else:
                    # Add time since opening
                    dt_end_open = datetime.datetime(
                        year=dt_end.year,
                        month=dt_end.month,
                        day=dt_end.day,
                        hour=self.worktiming[0],
                        minute=0)
                    worktime_in_seconds += (dt_end-dt_end_open).total_seconds()
        return int(worktime_in_seconds / 60)

    def is_weekend(self, datetime):
        """
        Returns True if datetime lands on a weekend.
        """
        for weekend in self.weekends:
            if datetime.isoweekday() == weekend:
                return True
        return False

In [ ]:
def stat_holiday_but_not_weekend(date1,date2):
    delta = date2-date1
    all_dates = []
    for i in range(delta.days+1):
        day = date2 + timedelta(days=i)
        all_dates.append(day)

    holiday_count=0
    for date in all_dates:
        stat_holiday = date in Canada_holidays
        if weekend_or_not(date)==False and stat_holiday==True:
            holiday_count+=10
        else:
            pass

    return holiday_count*60


def business_hours(date1,date2):
    if pd.isnull(date1) | pd.isnull(date2):
        return 0.0
    else:
        return (BusinessHours(date1,date2, worktiming=[7, 17], weekends=[6, 7]).getminutes()-stat_holiday_but_not_weekend(date1,date2))/60

In [ ]:
q = 1

In [ ]:
incidents_mod = incident_history

In [ ]:
incidents_mod['business hours'] = pd.Series(dtype='double')

In [ ]:
incidents_mod['business hours'] = 0.0

In [ ]:
for i in range(len(incidents_mod)):
    
    if (i >= len(incidents_mod) - 1):
        break
        
    elif incidents_mod['ticket_nmbr'][i][-1] != incidents_mod['ticket_nmbr'][q][-1]:
        q = q + 1
        continue
    
    elif incidents_mod['Priority'][i] == 1:
        incidents_mod.at[i, 'business hours'] = incidents_mod['TIME_IN_STATUS_BY_OWNER_HRS'][i]
        q = q + 1
        
    else:
        incidents_mod.at[i, 'business hours'] = business_hours(incidents_mod['CHANGE_DATE'][i], incidents_mod['CHANGE_DATE'][q])
        q = q + 1
    
    if i % 5000 == 0:
    	print(i)

In [ ]:
incidents_mod.to_csv('./incident_history_bh5.csv')  

In [ ]:
subset2 = incidents_mod[(incidents_mod["STATUS"]=="QUEUED")]

In [ ]:
subset2.head()

In [ ]:
subset2.to_csv('./incident_history_queued.csv')  

In [ ]:
subset2["TIME_IN_STATUS_BY_OWNER_HRS"].mean()